<a href="https://colab.research.google.com/github/auzaluis/upsa_mod_202501/blob/main/whisky/02_script_EDA_whisky.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import plotly.express as px

In [3]:
path = '/content/drive/MyDrive/Teaching/UPSA/2025.01/MOD/whisky/data-whisky.parquet'

In [4]:
df = pd.read_parquet(path)

**Lealtad**

In [19]:
df_lealtad = df['¿Cuál es la marca que más compra?'] \
.value_counts() \
.reset_index() \
.rename(columns = {
    '¿Cuál es la marca que más compra?': 'Marca',
    'count': 'n'
})

In [24]:
df_lealtad['Porcentaje'] = df_lealtad['n'] / df_lealtad['n'].sum()
df_lealtad['Porcentaje_txt'] = df_lealtad['Porcentaje'].apply(lambda x: str(round(x*100, 0)) + '%')

In [25]:
df_lealtad

,Marca,n,Porcentaje,Porcentaje_txt
0,Johnny Walker,43,0.524390,52.0%
1,Jack Daniel's,18,0.219512,22.0%
2,Chivas Regal,14,0.170732,17.0%
3,Old Parr,6,0.073171,7.0%
4,Ninguno,1,0.012195,1.0%


In [38]:
px.bar(
    df_lealtad,
    x = 'Marca',
    y = 'Porcentaje',
    text = 'Porcentaje_txt',
    title = '¿Cuál es la marca que más compra?',
    # color = 'Marca'
).update_layout(
    template = 'plotly_white',
    yaxis_showticklabels = False,
    yaxis_showgrid = False,
    xaxis_title = ''
)